<a href="https://colab.research.google.com/github/trhgquan/CS156/blob/main/sentiment_analysis/danh_gia_mon_hoc_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Importing libraries

In [4]:
# !pip install underthesea

In [58]:
import pandas as pd
import zipfile
from underthesea import word_tokenize

from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import classification_report
import joblib

## Extracting and creating .csv dataset

In [6]:
# !cp "/content/drive/MyDrive/Colab Notebooks/danh_gia_mon_hoc_rating/UIT-VSFC.zip" "/content/UIT-VSFC.zip"

def extracting_file():
    with zipfile.ZipFile('/content/UIT-VSFC.zip', 'r') as zfile:
        zfile.extractall('/content/data')

# extracting_file()

In [7]:
def create_dataframe(text, label):
    data = {'text': text, 'label': label}
    df = pd.DataFrame(data)
    return df

In [8]:
train_dataset = []
train_label = []

with open('/content/data/UIT-VSFC/train/sents.txt', 
          'r+', encoding = 'utf-8') as f:
    for line in f:
        train_dataset.append(line.rstrip())

with open('/content/data/UIT-VSFC/train/sentiments.txt', 
          'r+', encoding = 'utf-8') as f:
    for line in f:
        train_label.append(int(line.rstrip()))

print(len(train_dataset), len(train_label))

# Framing training dataset
train_data = create_dataframe(train_dataset, train_label)

11426 11426


In [9]:
print(train_data)

                                                    text  label
0                              slide giáo trình đầy đủ .      2
1         nhiệt tình giảng dạy , gần gũi với sinh viên .      2
2                   đi học đầy đủ full điểm chuyên cần .      0
3      chưa áp dụng công nghệ thông tin và các thiết ...      0
4      thầy giảng bài hay , có nhiều bài tập ví dụ ng...      2
...                                                  ...    ...
11421  chỉ vì môn game mà em học hai lần mà không qua...      0
11422                               em cảm ơn cô nhiều .      2
11423                           giao bài tập quá nhiều .      0
11424               giáo viên dạy dễ hiểu , nhiệt tình .      2
11425  gói gọn doubledot hay , tận tình , phù hợp với...      2

[11426 rows x 2 columns]


In [10]:
test_dataset = []
test_label = []

with open('/content/data/UIT-VSFC/test/sents.txt', 
          'r+', encoding = 'utf-8') as f:
    for line in f:
        test_dataset.append(line.rstrip())

with open('/content/data/UIT-VSFC/test/sentiments.txt', 
          'r+', encoding = 'utf-8') as f:
    for line in f:
        test_label.append(int(line.rstrip()))

test_data = create_dataframe(test_dataset, test_label)

In [70]:
print(test_data)

                                                   text  label
0                                nói tiếng anh lưu_loát      2
1                                giáo_viên rất vui_tính      2
2                                         cô max có tâm      2
3                              giảng bài thu_hút dí_dỏm      2
4     giáo_viên không giảng_dạy kiến_thức hướng_dẫn ...      0
...                                                 ...    ...
3161  các slide khó hiểu ngôn_ngữ trong slide phức_t...      0
3162                   giáo_viên giảng_dạy có tâm_huyết      2
3163                      chia_sẻ cho em nhiều điều hay      2
3164                                   em tiếp_thu chậm      0
3165  em có học ở một trung_tâm tiếng anh ở ngoài tr...      1

[3166 rows x 2 columns]


In [12]:
emb = None

def embedding(train, test):
    global emb
    emb = TfidfVectorizer(
        min_df = 5,
        max_df = 0.8,
        max_features = 3000,
        sublinear_tf = True
    )
    emb.fit(train)
    
    train = emb.transform(train)
    test = emb.transform(test)

    joblib.dump(emb, 'tfidf.pkl')
    return train, test

In [27]:
def tokenizer(sentence):
    return word_tokenize(sentence, format = 'text')

def standardlize(sentence):
    return sentence.replace('.', '').replace(',', '').replace('!', '').replace('  ', ' ')

In [28]:
train_data['text'] = train_data.text.apply(tokenizer)
train_data['text'] = train_data.text.apply(standardlize)
test_data['text'] = test_data.text.apply(tokenizer)
test_data['text'] = test_data.text.apply(standardlize)

In [29]:
X_train, X_test, y_train, y_test = train_data['text'], test_data['text'], train_data['label'], test_data['label']

In [32]:
X_train, X_test = embedding(X_train, X_test)

In [55]:
# model = svm.SVC(kernel = 'linear', C = 1)
model = LogisticRegression()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [59]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90      1409
           1       0.63      0.07      0.13       167
           2       0.91      0.91      0.91      1590

    accuracy                           0.88      3166
   macro avg       0.80      0.64      0.65      3166
weighted avg       0.87      0.88      0.87      3166



In [79]:
example = [
    ['thầy dạy rất dễ hiểu'],
    ['thầy toàn giải bài tập , không giảng gì cả'],
    ['cơ sở vật chất rất kém'],
    ['cảm ơn thầy rất nhiều'],
    ['thầy là phù thủy công nghệ'],
]

for e in example:
    test = emb.transform(e)
    print(model.predict(test))

[2]
[0]
[0]
[2]
[2]
